<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setting-up-imports" data-toc-modified-id="Setting-up-imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setting up imports</a></span></li><li><span><a href="#Setting-up-Constant-Hyperparameters" data-toc-modified-id="Setting-up-Constant-Hyperparameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setting up Constant Hyperparameters</a></span></li><li><span><a href="#Setting-up-Parameters-and-Functions-for-Training" data-toc-modified-id="Setting-up-Parameters-and-Functions-for-Training-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Setting up Parameters and Functions for Training</a></span><ul class="toc-item"><li><span><a href="#Hyperparameters-Search-Space" data-toc-modified-id="Hyperparameters-Search-Space-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Hyperparameters Search Space</a></span></li><li><span><a href="#Creating-the-training-function" data-toc-modified-id="Creating-the-training-function-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Creating the training function</a></span></li><li><span><a href="#Creating-the-evaluation-function" data-toc-modified-id="Creating-the-evaluation-function-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Creating the evaluation function</a></span></li></ul></li><li><span><a href="#Running-the-training" data-toc-modified-id="Running-the-training-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Running the training</a></span><ul class="toc-item"><li><span><a href="#Loading-data-for-training" data-toc-modified-id="Loading-data-for-training-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Loading data for training</a></span></li><li><span><a href="#Configuring-the-Tuner-with-a-Scheduler-and-a-Search-Algorithm" data-toc-modified-id="Configuring-the-Tuner-with-a-Scheduler-and-a-Search-Algorithm-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Configuring the Tuner with a Scheduler and a Search Algorithm</a></span></li><li><span><a href="#Running-the-Tuner" data-toc-modified-id="Running-the-Tuner-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Running the Tuner</a></span></li></ul></li><li><span><a href="#Evaluating-the-best-Results" data-toc-modified-id="Evaluating-the-best-Results-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Evaluating the best Results</a></span></li></ul></div>

# Setting up imports

In [1]:
import os
from itertools import product

import torch
from torch.nn import CrossEntropyLoss, Sequential
from torch.nn.functional import normalize
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from torchvision.transforms import CenterCrop, Resize, RandomCrop, GaussianBlur

import ray
from ray import tune
from ray.air import session, RunConfig, CheckpointConfig
from ray.air.checkpoint import Checkpoint
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.optuna import OptunaSearch
from ray.tune.search import ConcurrencyLimiter


from dataset import POCDataReader, data_augment_, POCDataset
from metrics import Metrics, EvaluationMetrics
from models import UNet, DeepCrack, SubUNet, DenSubUNet
from loss import *
from pipelines import InputPipeline, SequenceFilters, SumFilters
from pipelines.filters import *
from train import training_loop, validation_loop
from train_tqdm import evaluation_loop


# Setting up Constant Hyperparameters

In [ ]:
EPOCHS = 10
NUM_SAMPLES = 1

NUM_AUGMENT = 1

LOAD_DATA_ON_GPU = True
GPUS_PER_TRIAL = 1
CPUS_PER_TRIAL = 20

##### Selecting Cuda device

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Setting up Parameters and Functions for Training

## Hyperparameters Search Space

##### Preload Losses Functions

In [ ]:
pixel_loss_list = [
    CrossEntropyLoss(weight=torch.tensor([.65, .35])),
    FocalLoss(weight=torch.tensor([.65, .35]), gamma=1.4),
]

volume_loss_list = [
    JaccardLoss(),
    TverskyLoss(alpha=0.3, beta=0.7),
    FocalTverskyLoss(alpha=0.3, beta=0.7, gamma=1.4),
]

# loss_list = []
# for ploss in pixel_loss_list:
#     loss_list.append(PixelLoss(pixel_loss=ploss, volume_loss=None))
# for vloss in volume_loss_list:
#     loss_list.append(VolumeLoss(pixel_loss=None, volume_loss=vloss))
# for (ploss, vloss) in product(pixel_loss_list, volume_loss_list):
#     loss_list.append(CombinedLoss(loss1=ploss, loss2=vloss, ratio=0.3))
#     loss_list.append(BorderedLoss(border_loss=ploss, volume_loss=vloss, ratio=0.7))

##### Preload Pipeline

In [ ]:
filter_list = [normalize] #, invert]

layer_list = [
    None,
    SobelFilter(),
    LaplacianFilter(threshold=0.75),
    FrangiFilter(),
    SatoFilter(),
    SumFilters(FrangiFilter(), SatoFilter()),
    SkeletonFilter(SequenceFilters(SumFilters(FrangiFilter(), SatoFilter()), CrackBinaryFilter())),
]

# pipeline_list = []
# for f, l in product(filter_list, layer_list):
#     pipeline_list.append(InputPipeline(transformer=f, layer_transformer=l))

# no_layer_pip = InputPipeline(transformer=[normalize], layer_transformer=None)

##### Search Space

In [ ]:
search_space = {
    "Network": tune.grid_search([UNet, DeepCrack, SubUNet]), #tune.grid_search([UNet, DeepCrack, SubUNet, DenSubUNet]),
    "Optimizer": Adam,

    "Learning Rate": 1e-4, #tune.loguniform(1e-6, 1e-3),
    "Batch Size": 8,

    # "Loss Function": tune.grid_search(loss_list),
    "Loss Combiner": tune.grid_search([BorderedLoss, CombinedLoss]),
    "Loss Combiner_ratio": 0.75, #tune.uniform(0.6, 0.9),
    "Loss Volume": tune.grid_search(pixel_loss_list),
    "Loss Pixel": tune.grid_search(volume_loss_list),
    # "Loss Pixel_gamma": 1.4, #tune.uniform(1, 2),
    # "Loss Pixel_weight": 0.65, #tune.uniform(0, 0.75),

    "Negative Mining": False, #tune.choice([True, False]),
    "Smooth Labeling": False, #tune.choice([True, False]),

    "Pipe Filter": normalize,
    "Pipe Layer": tune.grid_search(layer_list),
    # "Pipe Layer_threshold": 0.75, #tune.loguniform(0.5, 5),
}

## Creating the training function

In [ ]:
def train(config, train_data, val_data):

    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    inpip = InputPipeline(
        transformer=config["Pipe Filter"],
        layer_transformer=config["Pipe Layer"])
    if LOAD_DATA_ON_GPU:
        inpip = inpip.to(device)

    train_dataset = POCDataset(
        train_data,
        transform=Sequential(RandomCrop(size=(256,256)), inpip),
        target_transform= Sequential(
            RandomCrop(size=(256,256)),
            GaussianBlur(kernel_size=3, sigma=0.7),
        ) if config["Smooth Labeling"] else RandomCrop(size=(256,256)),
        negative_mining=config["Negative Mining"],
        load_on_gpu=LOAD_DATA_ON_GPU)
    train_dataset.precompute_transform()

    if LOAD_DATA_ON_GPU:
        training_dataloader = DataLoader(
            train_dataset,
            batch_size=int(config["Batch Size"]),
            sampler=train_dataset.sampler,
            shuffle= True if train_dataset.sampler is None else None,
        )
    else:
        training_dataloader = DataLoader(
            train_dataset,
            batch_size=int(config["Batch Size"]),
            sampler=train_dataset.sampler,
            shuffle= True if train_dataset.sampler is None else None,
            num_workers=CPUS_PER_TRIAL//2,
            pin_memory=True,
            pin_memory_device=device)

    val_dataset = POCDataset(
        val_data, 
        transform=Sequential(RandomCrop(size=(256,256)), inpip),
        target_transform=RandomCrop(size=(256,256)),
        negative_mining=False,
        load_on_gpu=LOAD_DATA_ON_GPU)
    val_dataset.precompute_transform()
    
    if LOAD_DATA_ON_GPU:
        validation_dataloader = DataLoader(
            val_dataset,
            batch_size=int(config["Batch Size"]),
            shuffle=True)
    else:
        validation_dataloader = DataLoader(
            val_dataset,
            batch_size=int(config["Batch Size"]),
            shuffle=True,
            num_workers=CPUS_PER_TRIAL//2,
            pin_memory=True,
            pin_memory_device=device)

    model = config["Network"](n_channels=inpip.nb_channel, n_classes=2)
    if torch.cuda.is_available() and torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model.to(device)

    pixel_loss_fn = config["Loss Pixel"](
        weight=torch.tensor([config["Loss Pixel_weight"], 1.0 - config["Loss Pixel_weight"]]),
        gamma=config["Loss Pixel_gamma"])
    volume_loss_fn = config["Loss Volume"]()
    loss_fn = config["Loss Combiner"](
        border_loss=pixel_loss_fn,
        volume_loss=volume_loss_fn,
        ratio=config["Loss Combiner_ratio"]).to(device)

    optimizer = config["Optimizer"](model.parameters(), lr=config["Learning Rate"], betas=(0.9, 0.99))
    lr_scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state, scheduler_state = torch.load(os.path.join(loaded_checkpoint_dir, "checkpoint.pt"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
        lr_scheduler.load_state_dict(scheduler_state)

    train_metrics = Metrics(
        buffer_size=len(training_dataloader),
        mode="Training",
        hyperparam=config,
        device=device)

    val_metrics = Metrics(
        buffer_size=len(validation_dataloader),
        mode="Validation",
        hyperparam=config,
        device=device)


    for epoch in range(1, EPOCHS+1):  # loop over the dataset multiple times
        training_loop(epoch, training_dataloader, model, loss_fn, optimizer, lr_scheduler, train_metrics, device)
        validation_loop(epoch, validation_dataloader, model, loss_fn, val_metrics, device)

        # Here we save a checkpoint. It is automatically registered with
        # Ray Tune and can be accessed through `session.get_checkpoint()`
        # API in future iterations.
        os.makedirs("model", exist_ok=True)
        torch.save((model.state_dict(), optimizer.state_dict(), lr_scheduler.state_dict()), "model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("model")
        session.report(metrics=val_metrics.get_metrics(epoch), checkpoint=checkpoint)

    train_metrics.close_tensorboard()
    val_metrics.close_tensorboard()


## Creating the evaluation function

In [ ]:
def evaluate(test_data, result):

    device = "cuda:0" if torch.cuda.is_available() else "cpu"

    inpip = InputPipeline(
        transformer=result.config["Pipe Filter"],
        layer_transformer=result.config["Pipe Layer"](threshold=result.config["Pipe Layer_threshold"]))
    if LOAD_DATA_ON_GPU:
        inpip = inpip.to(device)

    test_dataset = POCDataset(
        test_data,
        transform=Sequential(RandomCrop(size=(256,256)), inpip),
        target_transform=RandomCrop(size=(256,256)),
        negative_mining=False,
        load_on_gpu=LOAD_DATA_ON_GPU)
    
    if LOAD_DATA_ON_GPU:
        evaluation_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)
    else:
        evaluation_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=2*CPUS_PER_TRIAL, pin_memory=True, pin_memory_device=device)

    best_trained_model = result.config["Network"](n_channels=inpip.nb_channel, n_classes=2).to(device)

    checkpoint_path = os.path.join(result.checkpoint.to_directory(), "checkpoint.pt")
    model_state, _, _ = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    test_metrics = EvaluationMetrics(
        buffer_size=len(evaluation_dataloader),
        hyperparam=result.config,
        epochs=result.metrics["Epoch"],
        device=device)

    evaluation_loop(dataloader=evaluation_dataloader, model=best_trained_model, metric=test_metrics, device=device)


# Running the training

## Loading data for training

In [ ]:
data_reader = POCDataReader(root_dir="../data/POC", load_on_gpu=False, verbose=True)
train_data, val_data, test_data = data_reader.split([0.7, 0.1, 0.2], seed=1234)
data_augment_(train_data, n=NUM_AUGMENT, load_on_gpu=False, verbose=True, seed=1234)

## Configuring the Tuner with a Scheduler and a Search Algorithm

In [ ]:
#scheduler = ASHAScheduler(max_t=EPOCHS, grace_period=2, reduction_factor=2)
# search_algo = HyperOptSearch()
# search_algo = OptunaSearch()

tune_config = tune.TuneConfig(
    metric="CrackIoU",
    mode="max",
    num_samples=NUM_SAMPLES,
#     scheduler=scheduler,
#     search_alg=search_algo,
    max_concurrent_trials=4,
)

tuner = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(train, train_data=train_data, val_data=val_data),
        resources={"cpu": CPUS_PER_TRIAL, "gpu": GPUS_PER_TRIAL}),
    tune_config=tune_config,
    param_space=search_space,
    run_config=RunConfig(
        local_dir="~/POC-Project/ray_results",
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute="CrackIoU",
            checkpoint_score_order="max",
            checkpoint_at_end=False)))

## Running the Tuner

In [ ]:
results = tuner.fit()

# Evaluating the best Results

In [ ]:
best_result = results.get_best_result(metric="CrackIoU", mode="max", scope="all")  # Get best result object
print("Best trial config: {}".format(best_result.config))
print("Best trial final validation loss: {}".format(best_result.metrics["Loss"]))
print("Best trial final validation CrackIoU: {}".format(best_result.metrics["CrackIoU"]))

for result in results:
    evaluate(test_data=test_data, result=result)